# Python for Data Analysis v2 | Notes_ Chapter_7

 本人以简书作者 SeanCheney 系列专题文章并结合原书为学习资源，记录个人笔记，仅作为知识记录及后期复习所用，原作者地址查看 [简书 SeanCheney](https://www.jianshu.com/u/130f76596b02)，如有错误，还望批评指教。——ZJ


>原作者：SeanCheney | [《利用 Python 进行数据分析·第2版》第7章 数据清洗和准备](https://www.jianshu.com/p/ac7bec000dad) | 來源：简书

>[Github:wesm](https://github.com/wesm/pydata-book) | [Github:中文 BrambleXu](https://github.com/BrambleXu/pydata-notebook)|
简书:[利用   Python   进行数据分析·第2版](https://www.jianshu.com/c/52882df3377a)

环境：   Python    3.6 

---

- 在数据分析和建模的过程中，相当多的时间要用在数据准备上：**加载、清理、转换以及重塑。**这些工作会占到分析师时间的 80% 或更多。有时，存储在文件和数据库中的数据的格式不适合某个特定的任务。
- pandas 和内置的 Python 标准库提供了一组高级的、灵活的、快速的工具，可以让你轻松地将数据规变为想要的格式。
- 在本章中，我会讨论**处理缺失数据、重复数据、字符串操作和其它分析数据转换的工具**。下一章，我会关注于用多种方法合并、重塑数据集。

## 7.1 处理缺失数据

在许多数据分析工作中，缺失数据是经常发生的。 pandas 的目标之一就是尽量轻松地处理缺失数据。例如， pandas 对象的所有描述性统计默认都不包括缺失数据。

缺失数据在 pandas 中呈现的方式有些不完美，但对于大多数用户可以保证功能正常。对于数值数据， pandas 使用浮点值 NaN（Not a Number）表示缺失数据。我们称其为哨兵值，可以方便的检测出来：

```
In [1]: import numpy as np

In [2]: import pandas as pd

In [3]: string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])

In [4]: string_data
Out[4]:
0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [5]: string_data.isnull()
Out[5]:
0    False
1    False
2     True
3    False
dtype: bool


```

- 在 pandas 中，我们采用了 R 语言中的惯用法，即将缺失值表示为 NA ，它表示不可用 not available。
- 在统计应用中， NA 数据可能是不存在的数据或者虽然存在，但是没有观察到（例如，数据采集中发生了问题）。
- 当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。

Python 内置的 None 值在对象数组中也可以作为 NA: 

```
In [6]: type(string_data)
Out[6]: pandas.core.series.Series

In [7]: string_data[0] = None

In [8]: string_data.isnull()
Out[8]:
0     True
1    False
2     True
3    False
dtype: bool


```
 pandas 项目中还在不断优化内部细节以更好处理缺失数据，像用户API功能，例如 pandas .isnull，去除了许多恼人的细节。表7-1列出了一些关于缺失数据处理的函数。

![](./images/7_1.png)

## 滤除缺失数据

- 过滤掉缺失数据的办法有很多种。你可以通过 `pandas.isnull`或布尔索引的手工方法，但 dropna 可能会更实用一些。
- 对于一个 Series ， **dropna 返回一个仅含非空数据和索引值的 Series **：

```
In [9]: from numpy import nan as NA

In [10]: data = pd.Series([1, NA, 3.5, NA, 7])

In [11]: data.dropna()
Out[11]:
0    1.0
2    3.5
4    7.0
dtype: float64

In [12]: data
Out[12]:
0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64


```

这等价于：

```
In [14]: data[data.notnull()]
Out[14]:
0    1.0
2    3.5
4    7.0
dtype: float64

```
- 而对于 DataFrame 对象，事情就有点复杂了。你可能希望丢弃全 NA 或含有 NA 的行或列。
- dropna 默认丢弃任何含有缺失值的行：


```
In [15]:  data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
    ...:                          [NA, NA, NA], [NA, 6.5, 3.]], index=['one', 't
    ...: wo', 'three', 'four'], columns=['first', 'second', 'third'])
    ...:
    ...:

In [16]: data
Out[16]:
       first  second  third
one      1.0     6.5    3.0
two      1.0     NaN    NaN
three    NaN     NaN    NaN
four     NaN     6.5    3.0

In [17]: cleaned  = data.dropna()

In [18]: cleaned
Out[18]:
     first  second  third
one    1.0     6.5    3.0


```

传入`how='all'`将只丢弃全为 NA 的那些行


```
In [19]: data.dropna(how='all')
Out[19]:
      first  second  third
one     1.0     6.5    3.0
two     1.0     NaN    NaN
four    NaN     6.5    3.0


```

用这种方式丢弃列(全是 NA 的列)，只需传入`axis=1`即可：


```
In [20]: data['forth'] = NA

In [21]: data
Out[21]:
       first  second  third  forth
one      1.0     6.5    3.0    NaN
two      1.0     NaN    NaN    NaN
three    NaN     NaN    NaN    NaN
four     NaN     6.5    3.0    NaN

In [22]: data.dropna(axis=1, how='all')
Out[22]:
       first  second  third
one      1.0     6.5    3.0
two      1.0     NaN    NaN
three    NaN     NaN    NaN
four     NaN     6.5    3.0


```

另一个滤除 DataFrame 行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用 thresh 参数实现此目的：


```
In [23]: df = pd.DataFrame(np.random.randn(7,3))

In [24]: df.iloc[:4, 1] = NA

In [25]: df
Out[25]:
          0         1         2
0 -0.052880       NaN  0.192669
1  0.440543       NaN -0.058121
2  0.297282       NaN -0.808425
3 -0.429874       NaN -0.965913
4  0.132290  0.251065  0.853049
5  1.190240 -1.118041 -0.075022
6  0.530970  0.033641 -0.473945

In [26]: df.iloc[:2, 2] = NA

In [27]: df
Out[27]:
          0         1         2
0 -0.052880       NaN       NaN
1  0.440543       NaN       NaN
2  0.297282       NaN -0.808425
3 -0.429874       NaN -0.965913
4  0.132290  0.251065  0.853049
5  1.190240 -1.118041 -0.075022
6  0.530970  0.033641 -0.473945

In [28]: df.dropna()
Out[28]:
         0         1         2
4  0.13229  0.251065  0.853049
5  1.19024 -1.118041 -0.075022
6  0.53097  0.033641 -0.473945

In [29]: df.dropna(thresh=2) # 索引 0 1 行 含有 NA 的去掉了
Out[29]:
          0         1         2
2  0.297282       NaN -0.808425
3 -0.429874       NaN -0.965913
4  0.132290  0.251065  0.853049
5  1.190240 -1.118041 -0.075022
6  0.530970  0.033641 -0.473945


```
## 填充缺失数据

- 你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补那些“空洞”。
- 对于大多数情况而言， fillna 方法是最主要的函数。通过一个常数调用 fillna 就会将缺失值替换为那个常数值：

```
In [30]: df.fillna(0)
Out[30]:
          0         1         2
0 -0.052880  0.000000  0.000000
1  0.440543  0.000000  0.000000
2  0.297282  0.000000 -0.808425
3 -0.429874  0.000000 -0.965913
4  0.132290  0.251065  0.853049
5  1.190240 -1.118041 -0.075022
6  0.530970  0.033641 -0.473945


```

若是通过一个字典调用 fillna ，就可以实现对不同的列填充不同的值：

```
In [31]: df.fillna({1: 0.5, 2: 0})
Out[31]:
          0         1         2
0 -0.052880  0.500000  0.000000
1  0.440543  0.500000  0.000000
2  0.297282  0.500000 -0.808425
3 -0.429874  0.500000 -0.965913
4  0.132290  0.251065  0.853049
5  1.190240 -1.118041 -0.075022
6  0.530970  0.033641 -0.473945

```

fillna 默认会返回新对象，但也可以对现有对象进行就地修改：

```
In [32]: _ = df.fillna(0, inplace= True)

In [33]: df
Out[33]:
          0         1         2
0 -0.052880  0.000000  0.000000
1  0.440543  0.000000  0.000000
2  0.297282  0.000000 -0.808425
3 -0.429874  0.000000 -0.965913
4  0.132290  0.251065  0.853049
5  1.190240 -1.118041 -0.075022
6  0.530970  0.033641 -0.473945

```

对 reindexing 有效的那些插值方法也可用于 fillna ：


```
In [34]: df = pd.DataFrame(np.random.randn(6,3))

In [35]: df.iloc[2:, 1] = NA

In [36]: df
Out[36]:
          0         1         2
0 -0.775110  0.000504  1.445061
1 -0.469458  0.727227 -0.166666
2  0.019312       NaN -0.915137
3 -1.477259       NaN  0.423064
4  1.620944       NaN  1.165360
5  0.388970       NaN  0.230785

In [37]: df.iloc[4:, 2] = NA

In [38]: df
Out[38]:
          0         1         2
0 -0.775110  0.000504  1.445061
1 -0.469458  0.727227 -0.166666
2  0.019312       NaN -0.915137
3 -1.477259       NaN  0.423064
4  1.620944       NaN       NaN
5  0.388970       NaN       NaN

In [39]: df.fillna(method='ffill')
Out[39]:
          0         1         2
0 -0.775110  0.000504  1.445061
1 -0.469458  0.727227 -0.166666
2  0.019312  0.727227 -0.915137
3 -1.477259  0.727227  0.423064
4  1.620944  0.727227  0.423064
5  0.388970  0.727227  0.423064

In [40]: df.fillna(method='ffill', limit=2)
Out[40]:
          0         1         2
0 -0.775110  0.000504  1.445061
1 -0.469458  0.727227 -0.166666
2  0.019312  0.727227 -0.915137
3 -1.477259  0.727227  0.423064
4  1.620944       NaN  0.423064
5  0.388970       NaN  0.423064


```

只要有些创新，你就可以利用 fillna 实现许多别的功能。比如说，你可以传入 Series 的平均值或中位数：

```
In [41]: data = pd.Series([1., NA, 3.5, NA, 7])

In [42]: data.fillna(data.mean())
Out[42]:
0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64



```

表7-2列出了 fillna 的参考。

![](./images/7_2.png)

```


```

## 7.2 数据转换

本章到目前为止介绍的都是数据的重排。另一类重要操作则是过滤、清理以及其他的转换工作。

## 移除重复数据

 DataFrame 中出现重复行有多种原因。下面就是一个例子：

```


```

 DataFrame 的duplicated方法返回一个布尔型 Series ，表示各行是否是重复行（前面出现过的行）：


```


```

还有一个与此相关的drop_duplicates方法，它会返回一个 DataFrame ，重复的数组会标为False：

```


```
这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断。假设我们还有一列值，且只希望根据k1列过滤重复项：



```


```

duplicated和drop_duplicates默认保留的是第一个出现的值组合。传入keep='last'则保留最后一个：

```


```

## 利用函数或映射进行数据转换

对于许多数据集，你可能希望根据数组、 Series 或 DataFrame 列中的值来实现转换工作。我们来看看下面这组有关肉类的数据：


```


```

假设你想要添加一列表示该肉类食物来源的动物类型。我们先编写一个不同肉类到动物的映射：

```


```

 Series 的map方法可以接受一个函数或含有映射关系的字典型对象，但是这里有一个小问题，即有些肉类的首字母大写了，而另一些则没有。因此，我们还需要使用 Series 的 str .lower方法，将各个值转换为小写：


```


```
我们也可以传入一个能够完成全部这些工作的函数：


```


```

使用map是一种实现元素级转换以及其他数据清理工作的便捷方式。

## 替换值

利用 fillna 方法填充缺失数据可以看做值替换的一种特殊情况。前面已经看到，map可用于修改对象的数据子集，而 replace 则提供了一种实现该功能的更简单、更灵活的方式。我们来看看下面这个 Series ：


```


```


-999这个值可能是一个表示缺失数据的标记值。要将其替换为 pandas 能够理解的 NA 值，我们可以利用 replace 来产生一个新的 Series （除非传入inplace=True）：


```


```

如果你希望一次性替换多个值，可以传入一个由待替换值组成的列表以及一个替换值：：


```


```

要让每个值有不同的替换值，可以传递一个替换列表：

```


```

传入的参数也可以是字典：


```


```
笔记：data. replace 方法与data. str . replace 不同，后者做的是字符串的元素级替换。我们会在后面学习 Series 的字符串方法。

## 重命名轴索引

跟 Series 中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新的不同标签的对象。轴还可以被就地修改，而无需新建一个数据结构。接下来看看下面这个简单的例子：




```


```


跟 Series 一样，轴索引也有一个map方法：

```


```

你可以将其赋值给index，这样就可以对 DataFrame 进行就地修改：

```


```

如果想要创建数据集的转换版（而不是修改原始数据），比较实用的方法是rename：


```


```

特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：

```


```

rename可以实现复制 DataFrame 并对其索引和列标签进行赋值。如果希望就地修改某个数据集，传入inplace=True即可：


```


```
## 离散化和面元划分

为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）。假设有一组人员数据，而你希望将它们划分为不同的年龄组：


```


```

接下来将这些数据划分为“18到25”、“26到35”、“35到60”以及“60以上”几个面元。要实现该功能，你需要使用 pandas 的cut函数：


```


```

 pandas 返回的是一个特殊的Categorical对象。结果展示了 pandas .cut划分的面元。你可以将其看做一组表示面元名称的字符串。它的底层含有一个表示不同分类名称的类型数组，以及一个codes属性中的年龄数据的标签：



```


```
pd.value_ count s(cats)是 pandas .cut结果的面元计数。

跟“区间”的数学符号一样，圆括号表示开端，而方括号则表示闭端（包括）。哪边是闭端可以通过right=False进行修改：



```


```
你可 以通过传递一个列表或数组到labels，设置自己的面元名称：


```


```
如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元。下面这个例子中，我们将一些均匀分布的数据分成四组：



```


```

选项precision=2，限定小数只有两位。

qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分。根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点。而qcut由于使用的是样本分位数，因此可以得到大小基本相等的面元：

```


```

与cut类似，你也可以传递自定义的分位数（0到1之间的数值，包含端点）：


```


```


本章稍后在讲解聚合和分组运算时会再次用到cut和qcut，因为这两个离散化函数对分位和分组分析非常重要。

检测和过滤异常值
过滤或变换异常值（outlier）在很大程度上就是运用数组运算。来看一个含有正态分布数据的 DataFrame 


```


```

假设你想要找出某列中绝对值大小超过3的值：


```


```
要选出全部含有“超过3或－3的值”的行，你可以在布尔型 DataFrame 中使用any方法：


```


```

根据这些条件，就可以对值进行设置。下面的代码可以将值限制在区间－3到3以内：


```


```
根据数据的值是正还是负，np.sign(data)可以生成1和-1：


```


```

## 排列和随机采样

利用numpy.random.permutation函数可以轻松实现对 Series 或 DataFrame 的列的排列工作（permuting，随机重排序）。通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组：


```


```
然后就可以在基于iloc的索引操作或take函数中使用该数组了：

```


```
如果不想用替换的方式选取随机子集，可以在 Series 和 DataFrame 上使用sample方法：


```


```
要通过替换的方式产生样本（允许重复选择），可以传递 replace =True到sample：


```


```

## 计算指标/哑变量

另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量”或“指标矩阵”。

如果 DataFrame 的某一列中含有k个不同的值，则可以派生出一个k列矩阵或 DataFrame （其值全为1和0）。 pandas 有一个 get_dummies 函数可以实现该功能（其实自己动手做一个也不难）。使用之前的一个 DataFrame 例子：


```


```
有时候，你可能想给指标 DataFrame 的列加上一个前缀，以便能够跟其他数据进行合并。 get_dummies 的prefix参数可以实现该功能：

```


```
如果 DataFrame 中的某行同属于多个分类，则事情就会有点复杂。看一下MovieLens 1M数据集，14章会更深入地研究它：


```


```
要为每个genre添加指标变量就需要做一些数据规整操作。首先，我们从数据集中抽取出不同的genre值：


```


```

现在有：



```


```
构建指标 DataFrame 的方法之一是从一个全零 DataFrame 开始：

```


```
现在，迭代每一部电影，并将dummies各行的条目设为1。要这么做，我们使用dummies.columns来计算每个类型的列索引：


```


```
然后，根据索引，使用.iloc设定值：


```


```

然后，和以前一样，再将其与movies合并起来：




```


```
笔记：对于很大的数据，用这种方式构建多成员指标变量就会变得非常慢。最好使用更低级的函数，将其写入NumPy数组，然后结果包装在 DataFrame 中。

一个对统计应用有用的秘诀是：结合 get_dummies 和诸如cut之类的离散化函数：

```


```
我们用numpy.random.seed，使这个例子具有确定性。本书后面会介绍 pandas . get_dummies 。

## 7.3 字符串操作

 Python 能够成为流行的数据处理语言，部分原因是其简单易用的字符串和文本处理功能。大部分文本运算都直接做成了字符串对象的内置方法。对于更为复杂的模式匹配和文本操作，则可能需要用到正则表达式。 pandas 对此进行了加强，它使你能够对整组数据应用字符串表达式和正则表达式，而且能处理烦人的缺失数据。

## 字符串对象方法

对于许多字符串处理和脚本应用，内置的字符串方法已经能够满足要求了。例如，以逗号分隔的字符串可以用split拆分成数段：



```


```

split常常与 str ip一起使用，以去除空白符（包括换行符）：

```


```
利用加法，可以将这些子字符串以双冒号分隔符的形式连接起来：


```


```
但这种方式并不是很实用。一种更快更符合 Python 风格的方式是，向字符串"::"的join方法传入一个列表或元组：

```


```

其它方法关注的是子串定位。检测子串的最佳方式是利用 Python 的in关键字，还可以使用index和find：

```


```

注意find和index的区别：如果找不到字符串，index将会引发一个异常（而不是返回－1）：

```


```
与此相关， count 可以返回指定子串的出现次数：

```


```

 replace 用于将指定模式替换为另一个模式。通过传入空字符串，它也常常用于删除模式：

```


```
表7-3列出了 Python 内置的字符串方法。

这些运算大部分都能使用正则表达式实现（马上就会看到）。

![](./images/7_3.png)
![](./images/7_3_1.png)

casefold 将字符转换为小写，并将任何特定区域的变量字符组合转换成一个通用的可比较形式。

## 正则表达式

正则表达式提供了一种灵活的在文本中搜索或匹配（通常比前者复杂）字符串模式的方式。正则表达式，常称作 regex ，是根据正则表达式语言编写的字符串。 Python 内置的re模块负责对字符串应用正则表达式。我将通过一些例子说明其使用方法。

笔记：正则表达式的编写技巧可以自成一章，超出了本书的范围。从网上和其它书可以找到许多非常不错的教程和参考资料。

re模块的函数可以分为三个大类：模式匹配、替换以及拆分。当然，它们之间是相辅相成的。一个 regex 描述了需要在文本中定位的一个模式，它可以用于许多目的。我们先来看一个简单的例子：假设我想要拆分一个字符串，分隔符为数量不定的一组空白符（制表符、空格、换行符等）。描述一个或多个空白符的 regex 是\s+：




```


```

调用re.split('\s+',text)时，正则表达式会先被编译，然后再在text上调用其split方法。你可以用re.compile自己编译 regex 以得到一个可重用的 regex 对象：

```


```

如果只希望得到匹配 regex 的所有模式，则可以使用findall方法：

```


```
笔记：如果想避免正则表达式中不需要的转义（\），则可以使用原始字符串字面量如r'C:\x'（也可以编写其等价式'C:\x'）。

如果打算对许多字符串应用同一条正则表达式，强烈建议通过re.compile创建 regex 对象。这样将可以节省大量的CPU时间。

match和search跟findall功能类似。findall返回的是字符串中所有的匹配项，而search则只返回第一个匹配项。match更加严格，它只匹配字符串的首部。来看一个小例子，假设我们有一段文本以及一条能够识别大部分电子邮件地址的正则表达式：


```


```

对text使用findall将得到一组电子邮件地址：

```


```
search返回的是文本中第一个电子邮件地址（以特殊的匹配项对象形式返回）。对于上面那个 regex ，匹配项对象只能告诉我们模式在原字符串中的起始和结束位置：


```


```

 regex .match则将返回 None ，因为它只匹配出现在字符串开头的模式：

```


```

相关的，sub方法可以将匹配到的模式替换为指定字符串，并返回所得到的新字符串：


```


```
假设你不仅想要找出电子邮件地址，还想将各个地址分成3个部分：用户名、域名以及域后缀。要实现此功能，只需将待分段的模式的各部分用圆括号包起来即可：

```


```
由这种修改过的正则表达式所产生的匹配项对象，可以通过其 groups 方法返回一个由模式各段组成的元组：


```


```
对于带有分组功能的模式，findall会返回一个元组列表：


```


```
sub还能通过诸如\1、\2之类的特殊符号访问各匹配项中的分组。符号\1对应第一个匹配的组，\2对应第二个匹配的组，以此类推：


```


```

 Python 中还有许多的正则表达式，但大部分都超出了本书的范围。表7-4是一个简要概括。

![](./images/7_4.png)

##  pandas 的矢量化字符串函数

清理待分析的散乱数据时，常常需要做一些字符串规整化工作。更为复杂的情况是，含有字符串的列有时还含有缺失数据：

```


```


通过data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在 NA （null）就会报错。为了解决这个问题， Series 有一些能够跳过 NA 值的面向数组方法，进行字符串操作。通过 Series 的 str 属性即可访问这些方法。例如，我们可以通过 str .contains检查各个电子邮件地址是否含有"gmail"：

```


```

也可以使用正则表达式，还可以加上任意re选项（如IGNORECASE）：

```


```


有两个办法可以实现矢量化的元素获取操作：要么使用 str .get，要么在 str 属性上使用索引：

```


```

要访问嵌入列表中的元素，我们可以传递索引到这两个函数中：


```


```

你可以利用这种方法对字符串进行截取：


```


```
表7-5介绍了更多的 pandas 字符串方法。



![](./images/7_5.png)


## 7.4 总结

高效的数据准备可以让你将更多的时间用于数据分析，花较少的时间用于准备工作，这样就可以极大地提高生产力。我们在本章中学习了许多工具，但覆盖并不全面。下一章，我们会学习 pandas 的聚合与分组。